# Treinamento das Redes VGGNet, ResNet e Inception

Código desenvolvido para o projeto de mestrado "Aplicação de Metodologia de Aprendizado Profundo para Identificação e Classificação de Budiões" - UNIFESP

## Declaração das bibliotecas

In [ ]:
import bentoml
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, Activation, Input, Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras import Sequential
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pickle
import os

## Leitura das imagens

Obtenção das imagens dos budiões já organizadas por diretório de treino e teste

In [ ]:
def get_images(pasta):
    exts = ['.PNG','.JPG','.JPEG','.TIFF','.GIF','.BMP']
    tot_images = 0
    for sf in [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))]:
        subdir = os.path.join(pasta,sf)
        tot_images = tot_images + len([name for name in os.listdir(subdir) if os.path.splitext(name)[1].upper() in exts])
    return tot_images

## Definição de Hiper parâmetros

Definição de variáveis (hiper parâmetros) usados na rede.

In [ ]:
#Arquitetura
architecture = 'VGG'
#architecture = 'RESNET'
#architecture = 'INCEPTION'

#Número de épocas (iterações) do treinamento
EPOCHS = 50;

#Serão gerados batches de 16 imagens (a rede vê 16 imagens de cada vez durante o treino). 
#Esse número é para facilitar o processamento conforme a memória do computador durante o treino, colocamos 16 não travar
BATCH_SIZE = 16; 

#Tamanho da imagem e 3 dimensões RGB
IMG_SIZE = (224,224,3); 
if architecture == 'INCEPTION':
    IMG_SIZE = (299,299,3);

#Seleção do Optimizador
optimizer_name = 'SGD'
#optimizer_name = 'Adam'

#Learning Rate
learning_rate = 0.01
#learning_rate = 0.0001
#learning_rate = 0.001
#learning_rate = 0.05

## Leitura dos nomes dos os arquivos para obtenção dos rótulos (labels)

Cada pasta e arquivo foi nomeado com a classe do budião (Scarus trispinosus_ADT, Scarus zelindae_IP, Sparisoma axillare_IP). Essa função faz a leitura dos nomes para uso (labels)

In [ ]:
# FUNÇÃO PARA LER O NOME DAS CLASSES
def get_labels(pasta):
    return [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))];

In [ ]:
# BUSCA OS NOMES DAS PASTAS
trainFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\train'
valFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\val'
labels = get_labels(trainFolder)
labels = np.array(labels);

# Organiza os labels em matriz e salva, para posterior uso em classificação
lb = LabelBinarizer();
labels = lb.fit_transform(labels);

# O arquivo .PICKLE é um arquivo que salva a configuração da rede
f = open('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\budioes_' + architecture + '_' + optimizer_name + '_' + str(learning_rate) + ".pickle", "wb")
f.write(pickle.dumps(lb));
f.close();

## Otimização da Memória do Computador

ImageDataGenerator = Carrega aos poucos as imagens em memória para fazer a leitura de cada uma, usado para gerenciar os recursos de hardware do computador (para não estourar a memória).

In [ ]:
if architecture == 'VGG' or architecture == 'RESNET':
    #ImageDataGenerator = objeto para buscar as imagens em uma pasta
    #Treino
    augTrain = ImageDataGenerator(rotation_range=20, width_shift_range = 0.1, height_shift_range = 0.1, 
                                  shear_range = 0.15, zoom_range = [1.0, 1.25], horizontal_flip=True, 
                                  fill_mode="nearest");
    #Validação
    augVal = ImageDataGenerator();

In [ ]:
if architecture == 'INCEPTION':
    #ImageDataGenerator = objeto para buscar as imagens em uma pasta
    #Treino
    #Quando carregar as imagens, será aplicada a função "preprocess_input" pré definida pela rede inception
    #O "preprocess_input" modifica os valores da imagem para facilitar o processamento.
    augTrain = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=20, width_shift_range = 0.1, height_shift_range = 0.1, 
                                  shear_range = 0.15, zoom_range = [1.0, 1.25], horizontal_flip=True, 
                                  fill_mode="nearest");
    #Validação
    augVal = ImageDataGenerator(preprocessing_function=preprocess_input);

# Arquitetura VGG

## Configuração da Rede convolucional (padrão da imagem) e transfer learning

É feita a declaração da rede VGG (convolution layer e pooling layer) e a aplicação do transfer learning usando a imagenet.

In [ ]:
if architecture == 'VGG':
    #Seleciona somente as camadas de convolução e retreina as de classificação
    baseModel = VGG16(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))
    #Como vamos usar imagenet, não faz sentido treinar a rede novamente pois já vamos usar o modelo treinado
    for layer in baseModel.layers:
        layer.trainable = False

## Configuração da Rede neural (classificação da imagem)

Configuração das 3 fully connected layers para classificação dos peixes budiões. Elas são organizadas em: primeira camada flatten, 3 conjuntos (dense, ativação e dropout) e a última camada dense (softmax).

In [ ]:
if architecture == 'VGG':
    #Include_Top configurado manualmente:
    #É preciso criar novas camadas (headModel) pra zerar os pesos da VGG
    headModel = baseModel.output
    headModel = Flatten(name='flatten')(headModel) #(headModel) é o mesmo que concatenar usando o .add
    headModel = Dense(4096,  name='fc1')(headModel)
    headModel = Activation('relu',  name='act_fc1')(headModel)
    #Recurso para evitar overfit (regularização), no caso, 20% dos parâmetros são zeros a cada iteração da rede. 
    #Isso força a rede a aprender outras formas de classificar
    headModel = Dropout(0.2) (headModel)
    headModel = Dense(2048,  name='fc2')(headModel)
    headModel = Activation('relu',  name='act_fc2')(headModel)
    headModel = Dropout(0.2) (headModel)
    headModel = Dense(512,  name='fc3')(headModel)
    headModel = Activation('relu',  name='act_fc3')(headModel)
    headModel = Dropout(0.2) (headModel)
    headModel = Dense(len(lb.classes_), activation="softmax", name='predictions')(headModel)
    #Junta tudo num modelo só
    model = Model(inputs=baseModel.input, outputs=headModel)
    #model.summary()

# Arquitetura ResNet

## Configuração da Rede convolucional (padrão da imagem) e transfer learning

É feita a declaração da rede ResNet (convolution layer e pooling layer) e a aplicação do transfer learning usando a imagenet.

In [ ]:
if architecture == 'RESNET':
    #Declaração da rede RESNET
    #Pega só as camadas de convolução e retreina as de classificação
    baseModel = ResNet101(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))
    #Como vamos usar imagenet, não faz sentido treinar a rede novamente pois já vamos usar o modelo treinado
    for layer in baseModel.layers:
        layer.trainable = False

## Configuração da Rede neural (classificação da imagem)

In [ ]:
if architecture == 'RESNET':
    #Include_Top configurado manualmente:
    #É preciso criar novas camadas (headModel) pra zerar os pesos da Resnet
    headModel = baseModel.output
    headModel = GlobalAveragePooling2D(name="avg_pool")(headModel)
    headModel = Dense(len(lb.classes_), activation="softmax", name='predictions')(headModel)
    #Junta tudo num modelo só
    model = Model(inputs=baseModel.input, outputs=headModel)
    #model.summary()

# Arquitetura Inception

## Configuração da Rede convolucional (padrão da imagem) e transfer learning

In [ ]:
if architecture == 'INCEPTION':
    #Declaração da rede INCEPTION V3
    #Utiliza somente as camadas de convolução e retreina as de classificação
    baseModel = InceptionV3(include_top=False, weights="imagenet", input_shape=(299,299,3), pooling='avg') 
    #Sempre utiliza 3 como parâmetro do input, pois corresponde ao tamanho do RGB da imagem
    #Como vamos usar imagenet, não faz sentido treinar a rede novamente pois já vamos usar o modelo treinado
    for layer in baseModel.layers:
        layer.trainable = False

## Configuração da Rede neural (classificação da imagem)

In [ ]:
if architecture == 'INCEPTION':
    #Include_Top configurado manualmente:
    #É preciso criar novas camadas (headModel) pra zerar os pesos da VGG
    headModel = baseModel.output
    headModel = Dense(len(lb.classes_), activation="softmax", name='predictions')(headModel)
    #Junta tudo num modelo só
    model = Model(inputs=baseModel.input, outputs=headModel)
    #model.summary()

# Execução do treinamento e validação dos resultados

## Aplicação do otimizador de acordo com a definição de hiperparâmetros

Os dois métodos de otimização utilizados são Stochastic Gradient Descent (SGD) ou Método do Gradiente Estocástico e Optimizer Adaptive Moment Estimation (ADAM) ou Estimativa de Momento Adaptativo.

In [ ]:
if optimizer_name == 'Adam':
    optimizer = Adam(learning_rate=learning_rate)

if optimizer_name == 'SGD':
    optimizer = SGD(learning_rate=learning_rate)

In [ ]:
#Reservando espaço de memória para a rede funcionar
#Ao final de cada época, será rodado o comando "callbacks"
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

## Preparação dos arquivos de treino e validação

Função para buscar e configurar as imagens de treino e validação

In [ ]:
#TREINO
trainGen = augTrain.flow_from_directory(
    trainFolder, #caminho da imagem
    class_mode="categorical", #o nome da pasta onde está a imagem será o nome da classe
    target_size=(IMG_SIZE[0], IMG_SIZE[1]), #tamanho da imagem a ser redimensionada
    color_mode="rgb", #a imagem terá 3 canais RGB
    shuffle=True, #vai embaralhar as imagens enquanto faz a leitura
    batch_size=BATCH_SIZE); #de quantas em quantas imagens será feita a leitura (tamanho do BATCH)

In [ ]:
#VALIDAÇÃO
valGen = augVal.flow_from_directory(
    valFolder,
    class_mode="categorical",
    target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE);

## Execução do treinamento e validação dos resultados

O comando model.fit executa o treinamento e validação conforme as configurações definidas anteriormente e salva o melhor resultado entre as épocas (callback) em um arquivo.

In [ ]:
#No nosso modelo, o callback irá salvar o melhor modelo entre as épocas (epochs) - função Model Checkpoint
callbacks = [
    #ReduceLROnPlateau(monitor = 'val_acc',factor=0.85, patience=10, min_lr=0.000001, verbose=1),
    ModelCheckpoint('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_' + architecture + '_' + optimizer_name + '_' + str(learning_rate) + 
                    '-ckpnt.model', save_best_only=True, monitor='val_accuracy', mode='max', verbose = 1)
]  

In [ ]:
#fit_generator vai de fato treinar a rede
trained_model = model.fit(trainGen, validation_data=valGen,
                        steps_per_epoch=get_images(trainFolder)//BATCH_SIZE,
                        validation_steps=get_images(valFolder) // BATCH_SIZE,
                        epochs = EPOCHS, callbacks=callbacks, verbose =1);
#Dividir o número de imagens pelo número de batchs para garantir que cada BATCH seja lido a cada época
#tanto no treino (steps_per_epoch) quanto na validação (validation_steps)
#verbose = dá output da rede a cada final de época

In [ ]:
#Salva o modelo (pesos + conexões entre os neurônios, ou seja, a estrutura da rede)
model.save('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_' + architecture + '_' + optimizer_name + '_' + str(learning_rate) + ".model");

## Salvar no formato bentoML

In [ ]:
#Modelo será exportado no formado do bentoML
bento_model = bentoml.keras.save_model('modelo_budioes_bentoML_' + architecture + '_' + optimizer_name + '_' + str(learning_rate) +'-ckpnt', model) 
    print(bento_model.tag)